### Neural network sizes

How many parameters are in our "alpha" model? Three "columns" with 2 convolution and pooling layers per column, feeding a dense layer, then concatenated and fed into another dense layer...

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/Users/perdue/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
imgw = 50
imgh = 50
depth = 1
nviews = 3  # instead of rgb depth, we have x,u,v views...

Real image dimensionality is (because u, v are interspersed with zero-columns to pad to the same size as x):

In [4]:
input_dim = imgw * imgh * 2
print input_dim

5000


Per branch (view):

In [5]:
# first layer has 32 3x3 filters
# weights should have shape (num_filters, num_input_channels, filter_rows, filter_columns)
filtw = 3
filth = 3
num_filters = 32
npars = filtw * filth * num_filters * depth

# biases? usually broadcast over depth? - yes...
# biases should have shape (num_filters,)
npars += num_filters

# new image size, with 2x2 maxpooling
imgw = (imgw - filtw + 1) / 2
imgh = (imgh - filth + 1) / 2
depth = num_filters

print npars, imgh, imgw, depth

320 24 24 32


In [6]:
# second layer has 32 3x3 filters
filtw = 3
filth = 3
num_filters = 32
npars += filtw * filth * num_filters * depth
npars += num_filters

# new image size, with 2x2 maxpooling
imgw = (imgw - filtw + 1) / 2
imgh = (imgh - filth + 1) / 2
depth = num_filters

print npars, imgh, imgw, depth

9568 11 11 32


In [7]:
# now, feed into fully connected layer
# dense layers should have size (num_inputs, num_units) with biases having size (num_units,)
npars += imgh * imgw * depth * 256 + 256

print npars

1001056


In [8]:
# 3 views
npars *= 3

print npars

3003168


In [9]:
# final dense layer of 11 outs
npars += 11 * (256 * 3) + 11

print npars

3011627


In [10]:
# datasize for float32 (4 bytes) in MB
size = npars * 4 / 1000000.

print size

12.046508


We observe a size on disk for this network of about 11 MB.

The size alone suggests how many params?

In [11]:
11e6 / 4.

2750000.0

In [12]:
(npars - 11e6 / 4.) / npars

0.08687231187660358

In [13]:
with np.load('models/lminervatriamese_model1457383290_alpha_v1r0.npz') as f:
    param_values = [f['arr_%d' % i] for i in range(len(f.files))]

In [14]:
len(param_values)

20

In [16]:
layer_names = {0:'conv-x', 1:'bias-x', 2:'conv-x', 3:'bias-x', 4:'dense-x', 5:'bias-x',
               6:'conv-u', 7:'bias-u', 8:'conv-u', 9:'bias-u', 10:'dense-u', 11:'bias-u',
               12:'conv-v', 13:'bias-v', 14:'conv-v', 15:'bias-v', 16:'dense-v', 17:'bias-v',
               18:'across-dense1', 19:'accros-bias1'
              }
for i, p in enumerate(param_values):
    print "{:02d}, {:15s}, {}".format(i, layer_names[i], np.shape(p))

00, conv-x         , (32, 1, 3, 3)
01, bias-x         , (32,)
02, conv-x         , (32, 32, 3, 3)
03, bias-x         , (32,)
04, dense-x        , (3872, 256)
05, bias-x         , (256,)
06, conv-u         , (32, 1, 3, 3)
07, bias-u         , (32,)
08, conv-u         , (32, 32, 3, 3)
09, bias-u         , (32,)
10, dense-u        , (3872, 256)
11, bias-u         , (256,)
12, conv-v         , (32, 1, 3, 3)
13, bias-v         , (32,)
14, conv-v         , (32, 32, 3, 3)
15, bias-v         , (32,)
16, dense-v        , (3872, 256)
17, bias-v         , (256,)
18, across-dense1  , (768, 11)
19, accros-bias1   , (11,)


In [15]:
for i, p in enumerate(param_values):
    print "{:02d}, {}".format(i, np.shape(p))

00, (32, 1, 3, 3)
01, (32,)
02, (32, 32, 3, 3)
03, (32,)
04, (3872, 256)
05, (256,)
06, (32, 1, 3, 3)
07, (32,)
08, (32, 32, 3, 3)
09, (32,)
10, (3872, 256)
11, (256,)
12, (32, 1, 3, 3)
13, (32,)
14, (32, 32, 3, 3)
15, (32,)
16, (3872, 256)
17, (256,)
18, (768, 11)
19, (11,)


3872 is the size of the image after two series of 3x3 convolution and 2x2 maxpooling (starting with 50x50) times a depth of 32 (the number of filters in the second convolutional layer). 

In [17]:
3 * (32 * 1 * 3 * 3 + 32 + 32 * 32 * 3 * 3 + 32 + 3872 * 256 + 256) + 768 * 11 + 11

3011627